## 공용 코드

In [4]:
# 파이썬
# ≥3.5 필수
import sys
assert sys.version_info >= (3, 5)

# 공통 모듈 임포트
import numpy as np
import pandas as pd
import os

# 깔끔한 그래프 출력을 위해 %matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

path = "c:/Windows/Fonts/malgun.ttf"

from matplotlib import font_manager, rc

import platform

if platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
    
mpl.rcParams['axes.unicode_minus'] = False
# Jupyter Notebook의 출력을 소수점 이하 3자리로 제한
%precision 3

import sklearn
assert sklearn.__version__ >= "0.20"

# 시드 고정
np.random.seed(21)

## 텍스트 파일에서 데이터 불러오기

### 세부 장르 가져오기

In [6]:
# 데이터 가져오기

origin_data = pd.read_excel('./3차_VOD_2308.xlsx')
sub_data = origin_data[['ct_cl','genre_of_ct_cl']]

In [10]:
# 세부 장르를 저장할 리스트
detail_genre_list = []

for data in sub_data.iterrows():
    # 데이터 구분
    ct_cl = data[1]['ct_cl']
    genre = data[1]['genre_of_ct_cl']
    
    # / 가 포함되어 있으면 파일 경로에 문제가 생기므로 & 로 대체
    if '/' in ct_cl:
        ct_cl = ct_cl.replace('/', '&')
    if '/' in genre:
        genre = genre.replace('/', '&')
    
    detail_genre = ct_cl + '-' + genre
    if detail_genre not in detail_genre_list:
        detail_genre_list.append(detail_genre)

In [ ]:
detail_genre_list

In [16]:
# 세부 장르 데이터 저장
output_file = open('./text files/detail_genre.txt', 'w', encoding = 'utf8')

for item in detail_genre_list:
    output_file.write(item + '\n')
    
output_file.close()

In [17]:
# 파일에서 세부 장르 데이터 가져오기

open_file = open('./text files/detail_genre.txt', 'r', encoding = 'utf8')
text = open_file.read()
open_file.close()

In [ ]:
# 파일에서 읽어온 데이터 확인
print(text)

In [20]:
data_list = text.split('\n')

In [ ]:
# data_list 의 마지막에 공백이 추가되므로 공백 제거
data_list = data_list[:-2]
# 확인
print(data_list)

### 세부 장르 별 요약 줄거리 데이터 가져오기

In [24]:
from konlpy.tag import Twitter
import nltk

In [ ]:
open_file = open('./text files/' + '영화-코미디' + '.txt', 'r', encoding = 'utf8')
text = open_file.read()

# 읽어온 텍스트 데이터에서 형태소를 분석하고 단어만 저장
spliter = Twitter()
nouns = spliter.nouns(text)
# nouns 는 각 단어를 str 형태로 리스트에 저장

#print(nouns) # 형태소 확인

open_file.close()

In [30]:
new_list = []
for item in nouns:
    if item not in new_list:
        new_list.append(item)

In [34]:
print(len(nouns))
print(len(new_list))

642
488


In [38]:
open_file = open('./text files/' + 'test' + '.txt', 'w', encoding = 'utf8')

for item in new_list:
    open_file.write(item + '\n')

open_file.close()

In [39]:
# 각 세부 장르의 줄거리가 저장된 파일에서 데이터 읽어오고 단어만 추출해서 저장 - 종합
for detail_genre in data_list:
    open_file = open('./text files/' + detail_genre + '.txt', 'r', encoding = 'utf8')
    text = open_file.read()
    
    # 읽어온 텍스트 데이터에서 단어만 추출
    spliter = Twitter()
    nouns = spliter.nouns(text)
    
    # 추출한 단어에서 중복을 제거
    new_list = []
    for item in nouns:
        if item not in new_list:
            new_list.append(item)
            
    # 중복이 없는 단어를 파일로 저장
    word_file = open('./text files/' + detail_genre + '_words.txt', 'w', encoding = 'utf8')
    for item in new_list:
        word_file.write(item + '\n')

    # 파일 닫기
    word_file.close()
    open_file.close()
    

In [40]:
word_dict_data = {}

# 각 세부 장르 별로 단어 리스트 생성하고 저장
for detail_genre in data_list:
    # 각 세부 장르에 대해 단어 데이터 가져오기
    open_file = open('./text files/' + detail_genre + '_words.txt', 'r', encoding = 'utf8')
    text = open_file.read()
    
    # 파일 닫기
    open_file.close()
    
    # 각 단어 분리하기
    data_list = text.split('\n')
    data_list = data_list[:-2] # 마지막의 공백 제외
    
    # 단어 리스트를 dict 형태로 저장
    word_dict_data[detail_genre] = data_list

In [ ]:
# 확인
# print(word_dict_data['TV 시사&교양-기타'])

### 문장의 단어 벡터화

In [42]:
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Okt

In [44]:
# 'TV 시사&교양-기타' 장르에 대해서 단어 벡터화 테스트

okt = Okt()
vectorizer = CountVectorizer(min_df = 0.05)
contents_tokens = word_dict_data['TV 시사&교양-기타']


# 벡터화를 위해 단어들을 가지고 문장 생성
contents_for_vect = []
sentence = ''
# 토큰 단위로 구분된 문장을 생성
for content in contents_tokens:
    sentence += ' ' + content

# 생성한 문장을 리스트에 추가
contents_for_vect.append(sentence)
# 확인
#print(contents_for_vect)

In [45]:
# 피처 벡터화

X = vectorizer.fit_transform(contents_for_vect)
# 피처 확인
print(vectorizer.get_feature_names_out())

['가게' '가까이' '가나' ... '희망' '히데' '히트']


In [50]:
print(X.toarray())
print(X.shape[1])
print(len(X.toarray()[0]))

[[1 1 1 ... 1 1 1]]
1411
1411


In [54]:
# 유사도를 측정할 줄거리 생성 - 9월 데이터 중 하나 활용
new_content = '밀실 안의 살인자, 정유정은 누구인가? 20대 또래 여성을 살해한 정유정의 정체와 범행 동기가 무엇인지에 대해 추적해 본다.'

# 문장 토큰화
spliter = Twitter()
words = spliter.nouns(new_content)
# 확인
# print(words)

# 테스트 문장을 피처 벡터화
new_content_vect = vectorizer.transform(words)
print(new_content_vect.toarray())
print(vectorizer.get_feature_names_out())
print(new_content_vect)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
['가게' '가까이' '가나' ... '희망' '히데' '히트']
  (0, 458)	1
  (2, 602)	1
  (4, 1067)	1
  (5, 253)	1
  (7, 821)	1
  (8, 605)	1
  (11, 1071)	1
  (12, 504)	1
  (14, 427)	1
  (16, 307)	1
  (17, 1201)	1


## 줄거리 거리 계산

In [82]:
# 샘플 데이터와 훈련 데이터 거리 확인

import scipy as sp

#거리 구해주는 함수 생성
def dist_raw(v1, v2):
    delta = v1 - v2
    return sp.linalg.norm(delta)


In [84]:
post_vec = X.getrow(0)
#print(post_vec.toarray())
#print(new_content_vect.toarray())

aaaa = new_content_vect.toarray()
bbbb = post_vec.toarray()

distance = dist_raw(aaaa, bbbb)
print("테스트 문장과의 거리: %.2f" %(distance))

테스트 문장과의 거리: 159.33


In [ ]:
#거리 구해주는 함수
def dist_raw(v1, v2):
    delta = v1 - v2
    return sp.linalg.norm(delta.toarray())

distance = dist_raw(new_content_vect, post_vec)